In [1]:
# Import all libraries at once
import csv, json, time
from datetime import date
import pickle
import os
import json
import numpy as np
from nrc_custom.CCUS_PostProcess import PostProcessing
from nrc_custom.CCUS_Active_Learning_Noise import ActiveLearning

In [2]:
# This block sets up the initial dictionary as part of the campaign. This is the object that will be updated throughout each run
experiment_name = 'rbnb3pxx'
number_runs = 50
exp_start = 0 # 0 is new campaign, other if continuing an existing campaign
dict = {}
dict[f'{experiment_name}'] = {}

#Pre populate the dictionary with the general data structure
for exp_count in range(number_runs):
    dict[f'{experiment_name}'][f'Test_{exp_count}'] = {'Depo':{},'Char':{},'Metric':{},'AL':{}}

# Create the folder if it doesn't exist based on the date of creation
date_format = date.today().strftime("%Y_%m_%d") # https://www.programiz.com/python-programming/datetime/current-datetime
root_path = f'C:/Users/Blackr/Documents/CCUS/MAPs/Initiate/{date_format}/'

if not os.path.exists(root_path): # Check if folder exists - if not, make it, else use existing folder
    os.makedirs(root_path)

# # Save the initial pickle file
with open(f'{root_path}{experiment_name}_saved_data.pkl', 'wb') as f:
            pickle.dump(dict, f)

In [4]:
with open(f'{root_path}{experiment_name}_saved_data.pkl', 'rb') as f:
    dict = pickle.load(f)   

In [6]:
dict

{'rbnb3pxx': {'Test_0': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_1': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_2': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_3': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_4': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_5': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_6': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_7': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_8': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_9': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_10': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_11': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_12': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_13': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_14': {'Depo': {}, 'Char': {}, 'Metric': {}, 'AL': {}},
  'Test_15': {'Depo': {}, 'Char': {}, 'Metric': {}, '

In [7]:
# The below starts the campaign, pulling in the correct information from the active learning protocol

for exp_count in range(exp_start,number_runs): # Define how many experiments the campaign will be
    # Update test name and dictionary
    test = f'Test_{exp_count}' # current experiment name - note in the potentiostat.py there is built in protection incase forget to change
    filename = f'{experiment_name}_{test}'
    print(f'For checking purposes the file name is: {filename}')

    # Initialize the active learning
    next_experiment = ActiveLearning(root_path, experiment_name, test, exp_count)
    
    # Initialize the experiment_update module
    experiment_update = PostProcessing(root_path, experiment_name, test)
    
    # Pick the first experiment randomly OR follow what was provided:
    if exp_count == 0:
        
        #Call in data from previous method to be able to access here:
        with open(f'{root_path}{experiment_name}_saved_data.pkl', 'rb') as f:
            dict = pickle.load(f)        
            
        next_experiment.determine_first_experiment(dict)
        #print(dict)
        
        X_choice = list(np.array(dict[f'{experiment_name}'][f'Test_{exp_count}']['AL']['X_sample'][0]))
        print(f'Initial experiment to run: {X_choice}')
        
        experiment_update.experiment_update(dict, exp_count, X_choice)
        px = X_choice[0]
        
    else: # If this is not the first experiment (ie. you need to continue the campaign after an error)
        
        #Call in data from previous method to be able to access here:
        with open(f'{root_path}{experiment_name}_saved_data.pkl', 'rb') as f:
            dict = pickle.load(f)

        #TODO: this will need to pull the last value from the previous test_name based on how it is stored, see Test_{i-1}
        X_choice = list(np.array(dict[f'{experiment_name}'][f'Test_{exp_count-1}']['AL']['X_sample'][-1]))
        
        print(f'Next experiment to run: {X_choice}')
        
        experiment_update.experiment_update(dict, exp_count, X_choice)
        px = X_choice[0]

    while True:
        next_exp = input('Understood what experiment to run next? Type -ok- to continue:')
        if next_exp == 'ok':
            break
    
    print("RUN EXPERIMENT")
    print("PROCESS DATA AND UPDATE ASSOCIATED OUTPUT MATRICES WITH NEW DATA")
        
    #exp_count += 1
    next_experiment.determine_next_experiment_random(dict)

For checking purposes the file name is: rbnb3pxx_Test_0
Initial experiment to run: [0.1, -0.4, 32.5]
Understood what experiment to run next? Type -ok- to continue:ok
RUN EXPERIMENT
PROCESS DATA AND UPDATE ASSOCIATED OUTPUT MATRICES WITH NEW DATA
the previous X_samples matrix imported and is currently active is [[ 0.1 -0.4 32.5]]
This is the random index chosen for this run = [513]
New candidates composition to be added to trainset: tensor([[ 1.0000, -0.3000, 49.0000]], dtype=torch.float64)
For checking purposes the file name is: rbnb3pxx_Test_1
Next experiment to run: [1.0, -0.3, 49.0]
Understood what experiment to run next? Type -ok- to continue:ok
RUN EXPERIMENT
PROCESS DATA AND UPDATE ASSOCIATED OUTPUT MATRICES WITH NEW DATA
the previous X_samples matrix imported and is currently active is [[ 0.1 -0.4 32.5]
 [ 1.  -0.3 49. ]]
This is the random index chosen for this run = [360]
New candidates composition to be added to trainset: tensor([[ 0.6000,  0.5000, 54.5000]], dtype=torch.floa

KeyboardInterrupt: Interrupted by user

In [8]:
import numpy as np
import torch
delete_me = np.array([0,0,0]) # Need to just make a dummy matrix first of the row number of columns
a1 = np.round(np.linspace(0,1,num = 11), decimals = 1)
a2 = np.round(np.linspace(-0.5,0.5,num = 11), decimals = 1)
a3 = np.round(np.linspace(5,60,num = 11), decimals = 1) 
for i in a1:
    for j in a2:
        for k in a3:
            add = np.array([i,j,k])
            delete_me = np.vstack((delete_me,add))
X_matrix = np.delete(delete_me, (0), axis=0)
X_matrix = torch.tensor(X_matrix)

In [33]:
#TODO - Update the appropriate pump dispension information and Depo.json file to take into account the new information



# Update .json files with correct information:
# Need to update char_1 & char_2 status to 'incomplete', and depo to incomplete and update parameters
# Need to update the pump dispension information

# Class exp_update:

# Import files:
with open('master_runs_depo.json') as f:
   depo_json = json.load(f)

with open('master_runs_char1.json') as f:
   char_1_json = json.load(f)

with open('master_runs_char2.json') as f:
   char_2_json = json.load(f)

# Update the run status of all
char1_json['Runs'][0]['status'] = 'incomplete'
char2_json['Runs'][0]['status'] = 'incomplete'
depo_json['Runs'][0]['status'] = 'incomplete'
char1_json['Runs'][0]['expID'] = exp_count
char2_json['Runs'][0]['expID'] = exp_count
depo_json['Runs'][0]['expID'] = exp_count

# Update the new experimental parameters in depo.json and the pump parameters
# Assume X_sample matric is set ip as [0] = Au[], [1] = voltage, [2] = time
# Update the 
depo_json['Runs'][0]['Techniques'][1]['params']['voltage_step'] = X_choice[1]
depo_json['Runs'][0]['Techniques'][1]['params']['duration_step'] = X_choice[2]

#To update the pump:
px = []
px.append(DispenseProcedure(1,X_choice[0],1,0)) # pumpnum,target_wgt,carouselsink,carouseldump
    
# Save the pickle file
with open('master_runs_depo.json') as f:
    json.dump(depo_json, f)
with open('master_runs_char1.json') as f:
    json.dump(char1_json, f)
with open('master_runs_char2.json') as f:
    json.dump(char2_json, f)

In [56]:
        #MAKE THIS INTO A FUNCTION#################################################################
        # Import files:
        with open('master_runs_depo.json') as f:
           depo_json = json.load(f)

        with open('master_runs_char1.json') as f:
           char_1_json = json.load(f)

        with open('master_runs_char2.json') as f:
           char_2_json = json.load(f)

        # Update the run status of all
        char1_json['Runs'][0]['status'] = 'incomplete'
        char2_json['Runs'][0]['status'] = 'incomplete'
        depo_json['Runs'][0]['status'] = 'incomplete'
        char1_json['Runs'][0]['expID'] = exp_count
        char2_json['Runs'][0]['expID'] = exp_count
        depo_json['Runs'][0]['expID'] = exp_count

        # Update the new experimental parameters in depo.json and the pump parameters
        # Assume X_sample matric is set ip as [0] = Au[], [1] = voltage, [2] = time
        # Update the 
        depo_json['Runs'][0]['Techniques'][1]['params']['voltage_step'] = X_choice[1]
        depo_json['Runs'][0]['Techniques'][1]['params']['duration_step'] = X_choice[2]

        #To update the pump. global variable should have it carry-over into the main text:
        global px
        px = []
        px.append(DispenseProcedure(1,X_choice[0],1,0)) # pumpnum,target_wgt,carouselsink,carouseldump

        # Save the pickle file
        with open('master_runs_depo.json') as f:
            json.dump(depo_json, f)
        with open('master_runs_char1.json') as f:
            json.dump(char1_json, f)
        with open('master_runs_char2.json') as f:
            json.dump(char2_json, f)
        ##################################################################

150

In [49]:
depo_json['Runs'][0]

{'status': 'incomplete',
 'expID': 150,
 'channel': 1,
 'user': 'Fonsecacd',
 'labID': 'NRC',
 'material_info': 'Na',
 'Techniques': [{'params': {'rest_time_T': 30,
    'record_every_dT': 0.5,
    'record_every_dE': 0.1,
    'E_range': 'E_RANGE_5V'},
   'TechID': 'OCV'},
  {'params': {'vs_initial': [False],
    'voltage_step': [3],
    'duration_step': [30],
    'n_cycles': 0,
    'record_every_di': 0.1,
    'record_every_dt': 0.1,
    'I_range': 'I_RANGE_10mA',
    'E_range': 'E_RANGE_5V',
    'bandwidth': 'BW_5'},
   'TechID': 'CA'}]}

In [64]:
a = 4
b = 0
def try_something(a):
    global b
    b = []
    c = a + a
    b.append(c)
    print(b)
try_something(a)
print(b)

[8]
[8]


In [1]:
# Seem to have a big problem when putting the dispense class as an external library and 'c9' not being defined. 
# Calling the class here seems fine, but as an external library, run into the problem????
# **********************************************************************************************
# Define statements

CAROUSEL_ROT = 4 # rotary
CAROUSEL_Z = 6 # elevation
# **********************************  FLUID DISPENSATION FUNCTIONS END ****************************************

# **********************************************************************************************
# Class 
class DispenseProcedure:
    # **********************************************************************************************
    # Constructor definition
    def __init__(self,pumpnum,target_wgt,carouselsink,carouseldump):
        
        self.pumpnum = pumpnum # 9
        self.target_wgt = target_wgt
        
        self.carouselsink = carouselsink #3 - Count from 1.  Sink position changes depending on which sink port we want to dispense.
        self.carouseldump = carouseldump #2 - Count from 1.  Dump position does not change 

    # **********************************************************************************************
    # Function definition
    
    # Desc : Reports the settings going into the object made by the Constructor
    def ReportSettings(self):
        print("Object Created : Pump Number {0} is active and Target Weight {1} is requested.".format(self.p1.pumpnum,self.p1.target_wgt))

    # Desc : Moves carousel's axes (rotation and elevation) as defined by incoming variables
    def move_carousel(self, rot_deg, z_mm, vel=None, accel=None):
        #self.rot_deg = rot_deg
        #self.z_mm = z_mm
        #self.vel = vel
        #self.accel = accel
        
        if ((rot_deg > 330) or (z_mm > 160)):
            return
        c9.move_axis(CAROUSEL_Z, 0, vel, accel)
        c9.move_axis(CAROUSEL_ROT, int(rot_deg*(51000/360)), vel, accel)
        c9.move_axis(CAROUSEL_Z, int(z_mm*(40000/160)), vel, accel) # vel = counts/sec, accel = counts/sec2

    # Desc : Obtain stable weight of liquid in vial.  Note - mass balance has to be zeroed first.
    def measure_weight(self):
        c9.clear_scale()
        c9.delay(2) # delay enables any drops travelling down the tube fall into the vial
        st = c9.read_steady_scale()
        print(st)
        index = 0
        weight = 0
        
        c9.delay(2)
        weight = st
            
        print("\nFinal stable weight : ", weight)
        return weight

    # Desc : Rotates carousel port to positioned defined by incoming variable.
    #        Checks should be added to function below and tested to ensure port numbers range between 1 and 7 (i.e. not out of range)
    def set_carousel_port(self, pos):
        # pos represents the position of the carousel dispenser from 1 to 7
        self.move_carousel((pos * 45) + 3, 127) # note : the +3 is for the azimuth offset error.  (max vals are 330.0 and 155)
        
    # Desc : Returns carousel axes positions to its home.
    def home_carousel_axis(self):
        # base - rotary (4)
        # top - up/down aka elevation (6)
        c9.home_axis(4)
        c9.home_axis(6)

    # Desc : Clears and zeroes the mass balance.    
    def zero_weigh_scale(self):
        c9.delay(1)
        c9.clear_scale()
        c9.delay(1)
        c9.zero_scale()
    
#     def catalyst_procedure(self): # blank holder for catalyst procedure
#         pass

# **********************************************************************************************
    def prime_pumps(self):
        # pos represents the position of the carousel dispenser from 1 to 7
        p1.set_carousel_port(p1.carouseldump) 

        # --------------------------------------------------------------------
        # Prime the pump - this is on the Source side

        # First, set the pump and valve to the default valve position
        # NOTE : Default valve position has the valve to the source tank open.
        c9.set_pump_valve(p1.pumpnum,0)

        c9.delay(1)
        c9.home_pump(p1.pumpnum)

        # home the pump (again?!)
        c9.delay(1)
        c9.set_pump_valve(p1.pumpnum,0)

        # suck up X ml from vial
        c9.delay(1)
        c9.aspirate_ml(p1.pumpnum,1) # 1 was 0.5
        c9.delay(2) # almost certainly need this delay for the fluid to be sucked up fully with the negative pump pressure

        # set the pump and switch the valve to the dispense position
        c9.set_pump_valve(p1.pumpnum,1)

        # dispense X ml from vial
        c9.delay(1)
        c9.dispense_ml(p1.pumpnum,1)
        c9.delay(2) # need this delay as there are still some drops falling as the tube dispenses the fluid with positive pump pressure

        # set the pump and switch the valve back to default valve position
        c9.set_pump_valve(p1.pumpnum,0)
        c9.delay(1)
        
    def catalyst_procedure(self, dispense_num):
        # Move Carousel to position where it will Dispense into Vial
        # dispense_num is for tracking and recording the weights
        p1.set_carousel_port(p1.carouselsink)

        # --------------------------------------------------------------------
        # Have the pump suck up a full cylinder of fluid from Source

        # First, set the pump and valve to the default valve position (just in case)
        # NOTE : Default valve position has the valve to the source tank open.
        c9.set_pump_valve(p1.pumpnum,0)
        c9.delay(1)

        # suck up X ml from source
        c9.aspirate_ml(p1.pumpnum,1) # 1 was 0.5
        c9.delay(2) # almost certainly need this delay for the fluid to be sucked up fully with the negative pump pressure

        # set the pump and switch the valve to the dispense position
        c9.set_pump_valve(p1.pumpnum,1)
        c9.delay(1)

        # -------------------------------------------------------------

        # Zero the weight scale with the empty vial
        # We are about to measure (by weight) how much liquid we have dispensed  
        p1.zero_weigh_scale()

        # -------------------------------------------------------------
        # Measuring weight of (incrementally) dispensed fluid on mass balance in a closed feedback loop manner till it hits the weight target.
        # Dispensation quantity of fluid from the pump is stepped down as mass balance approaches its target of 0.050 mL.
        # For distilled water, there is a 1:1 relationship between fluid weight and fluid volume (i.e. 1.000 g = 1.000 mL).
        # Would need to calculate the ratio for fluids of different chemicals accordingly using their concentration (i.e. molar mass).
        # If the fluid pump has already dispensed more than 3/4 of the quantity of fluid in the cylinder, the pump is refilled (re-primed).

        wgt = p1.measure_weight()
        
        # Below is to ensure the scale actually zeros as have run into problems with this. 0.003 is arbitrary here - just something to catch the big numbers
        num_check = 0 # To break the loop in case it can't zero after 5 attempts
        while abs(wgt) > 0.003: 
            p1.zero_weigh_scale()
            wgt = p1.measure_weight()
            print(f're-zeroing scale {num_check} times')
            num_check = num_check+1
            if num_check > 5:
                print('breaking the loop and couldn not fully re-zero (check > 5)')
                break
            
            
        addon_disp = 0
        # p1.target_wgt = 1.000  # use this if you want to over-ride the setting made in the constructor (way up above)
        dispvar = 0.025  # by default, dispense this much mL (the smallest displacement as seen below in the while)

        # we are attempting to hit the targeted weight within 0.01 of the vial's weight reading
        # whereupon we stop dispensing fluid.
        while ((p1.target_wgt - wgt) > 0.005):
            # dispense X ml from vial    
            if (p1.target_wgt - wgt > 0.50):
                dispvar = 0.45     
            elif (p1.target_wgt - wgt > 0.35):
                dispvar = 0.32   
            elif (p1.target_wgt - wgt > 0.15):
                dispvar = 0.12
            elif (p1.target_wgt - wgt > 0.05):
                dispvar = 0.04
            elif (p1.target_wgt - wgt > 0.02):
                dispvar = 0.025
            else:
                dispvar = 0.025

            addon_disp += dispvar

            c9.dispense_ml(p1.pumpnum,dispvar)

            wgt = p1.measure_weight()
            #print(wgt)

            # if pump has dispensed more than 3/4 of its volume, dump the remainder and refill (re-prime) the pump
            if(addon_disp > 0.9):     
                p1.set_carousel_port(p1.carouseldump) # move to the position of the fluid dump receptical

                c9.delay(1)
                c9.home_pump(p1.pumpnum)

                c9.delay(1)
                c9.set_pump_valve(p1.pumpnum,0)

                # suck up X ml from source
                c9.aspirate_ml(p1.pumpnum,1) # fill full cylinder
                c9.delay(2) # almost certainly need this delay for the fluid to be sucked up fully with the negative pump pressure

                # move carousel back over vial
                p1.set_carousel_port(p1.carouselsink)

                # return back to dispensing valve position
                c9.delay(2)
                c9.set_pump_valve(p1.pumpnum,1)
                c9.delay(2)            

                addon_disp = 0 # reset the adddon_disp variable since we have filled up the cylinder to 100%
                # we are ready to continue

        print(wgt)
        dict[f'{experiment_name}'][f'Test_{exp_count}']['Metric']['X_mass'][dispense_num] = wgt # Add final weight to dictionary for tracking

        # --------------------------------------------------------------------

        # Set the pump and switch the valve back to default valve position.
        c9.delay(0.5)
        c9.set_pump_valve(p1.pumpnum,0)
        c9.delay(2)

        # --------------------------------------------------------------------

        # Move to the fluid dump receptical to dump remainder fluid in the pump's cylinder.
        p1.set_carousel_port(p1.carouseldump)

        # --------------------------------------------------------------------

        # Perform the dump of fluid into the fluid dump receiptical.
        c9.home_pump(p1.pumpnum)
        c9.delay(3)

        # --------------------------------------------------------------------

        # Return the carousel rotation and elevation axis to its home position.
        # p1.home_carousel_axis()

    # --------------------------------------------------------------------
    # END